# XGB 베이스라인

In [2]:
import pandas as pd
import os
import random
import numpy as np

import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold
import wandb 
from wandb.xgboost import wandb_callback
from tqdm import tqdm
from xgboost import XGBClassifier
from xgboost import plot_importance

ImportError: this version of pandas is incompatible with numpy < 1.20.3
your numpy version is 1.19.2.
Please upgrade numpy to >= 1.20.3 to use this pandas version

In [2]:
pd.options.display.max_columns = 100

## 1. 데이터 로딩

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmscornjs (recsys-10). Use `wandb login --relogin` to force relogin


True

In [4]:
data_dir = '/opt/ml/input/data/' # 경로는 상황에 맞춰서 수정해주세요!

# LOAD TRAINDATA
csv_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
train_data = pd.read_csv(csv_file_path)

# LOAD TESTDATA
test_csv_file_path = os.path.join(data_dir, 'test_data.csv')
test_data = pd.read_csv(test_csv_file_path)
# test_data['answerCode'] = test_data['answerCode'].replace(-1, np.nan)

# # LEAVE LAST INTERACTION ONLY
# train_df = test_df[test_df['userID'] == test_df['userID'].shift(-1)]
# test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# df = pd.concat([df, train_df], ignore_index=True)

## 2. Feature Engineering

In [6]:
def feature_engineering(train, test):
    ## train, test data concat
    train['train'] = 1
    test['train'] = 0
    total = pd.concat([train, test], ignore_index=True)
    # total['answerCode'] = total['answerCode'].fillna(0.5)
    total = elo(total)
    
    ## 유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    total.sort_values(by=['userID','Timestamp'], inplace=True)
    total['Timestamp'] = pd.to_datetime(total['Timestamp'])
    
    total['assessmentItemID'] = total['assessmentItemID'].astype('category')
    total['KnowledgeTag'] = total['KnowledgeTag'].astype('category')

    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    total['user_correct_answer'] = total.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    total['user_total_answer'] = total.groupby('userID')['answerCode'].cumcount()
    total['user_acc'] = total['user_correct_answer']/total['user_total_answer']

    total['Bigcat'] = total['testId'].str[2]
    total['Bigcat'] = total['Bigcat'].astype('category')
    total['Bigcat_avg'] = total.groupby('Bigcat')['answerCode'].transform('mean')
    
    # total['past_count'] = total.groupby('userID').cumcount()
    
    diff = total.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())
    total['elapsed'] = diff

    total['item_avg'] = total.groupby('assessmentItemID')['answerCode'].transform('mean')
    total['answer_cnt'] = total.groupby(['userID'])['answerCode'].rolling(5, min_periods=1).sum().shift(1).values
    total['answer_cnt'] = total['answer_cnt'].fillna(0)

    categories = ["assessmentItemID",'KnowledgeTag',"Bigcat"]

    # # label encode your categorical columns
    le = LabelEncoder()
    for category in categories:
        total[category] = le.fit_transform(total[category])
    # total['Bigcat'] = le.fit_transform(total[category])

    ## train, test 나누어서 return
    train_df = total[total['train']==1]
    test_df = total[total['train']==0]
    test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]
    
    
    return train_df, test_df

In [7]:
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [8]:
def elo(df):
    def get_new_theta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return theta + learning_rate_theta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def get_new_beta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return beta - learning_rate_beta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def learning_rate_theta(nb_answers):
        return max(0.3 / (1 + 0.01 * nb_answers), 0.04)

    def learning_rate_beta(nb_answers):
        return 1 / (1 + 0.05 * nb_answers)

    def probability_of_good_answer(theta, beta, left_asymptote):
        return left_asymptote + (1 - left_asymptote) * sigmoid(theta - beta)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def estimate_parameters(answers_df, granularity_feature_name="assessmentItemID"):
        item_parameters = {
            granularity_feature_value: {"beta": 0, "nb_answers": 0}
            for granularity_feature_value in np.unique(
                answers_df[granularity_feature_name]
            )
        }
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        print("Parameter estimation is starting...", flush=True)

        for student_id, item_id, left_asymptote, answered_correctly in tqdm(
            zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            ),
            total=len(answers_df),
        ):
            theta = student_parameters[student_id]["theta"]
            beta = item_parameters[item_id]["beta"]

            item_parameters[item_id]["beta"] = get_new_beta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                item_parameters[item_id]["nb_answers"],
            )
            student_parameters[student_id]["theta"] = get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

            item_parameters[item_id]["nb_answers"] += 1
            student_parameters[student_id]["nb_answers"] += 1

        print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        return student_parameters, item_parameters

    def gou_func(theta, beta):
        return 1 / (1 + np.exp(-(theta - beta)))

    df["left_asymptote"] = 0

    print(f"Dataset of shape {df.shape}")
    print(f"Columns are {list(df.columns)}")

    student_parameters, item_parameters = estimate_parameters(df)

    prob = [
        gou_func(student_parameters[student]["theta"], item_parameters[item]["beta"])
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
    ]

    df["elo"] = prob

    return df

## 3. Train/Test 데이터 셋 분리

In [9]:
# wandb.init(project="LGBM", config= config)

df, submission = feature_engineering(train_data, test_data)
# df = elo(df)

# 유저별 분리
train, test = custom_train_test_split(df)
print(train.columns)

# 사용할 Feature 설정
# FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
#          'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum', 'elapsed','Bigcat','smallcat']
# FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
#          'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum', 'elapsed', 'Bigcat', 'smallcat', 'item_num', 'item_seq', 'solved_time_shift']
# FEATS = ['KnowledgeTag', 'same_item_cnt', 'user_avg', 'item_avg', 'test_avg', 'tag_avg', 'user_time_avg', 'item_time_avg',
#        'test_time_avg', 'tag_time_avg', 'user_current_avg', 'user_current_time_avg', 'hour', 'item_num', 'Bigcat','Bigcat_avg']
# FEATS = ['assessmentItemID', 'KnowledgeTag','user_acc','Bigcat_avg','elapsed','elo']
# FEATS = ['KnowledgeTag', 'same_item_cnt', 'user_avg', 'item_avg', 'test_avg', 'tag_avg']
FEATS = ['assessmentItemID', 'KnowledgeTag','Bigcat_avg','item_avg','elapsed','elo','answer_cnt']

# X, y 값 분리
y_train = train['answerCode']
train = train.drop(['answerCode'], axis=1)

y_test = test['answerCode']
test = test.drop(['answerCode'], axis=1)

Dataset of shape (2526700, 8)
Columns are ['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'train', 'left_asymptote']
Parameter estimation is starting...


100%|██████████| 2526700/2526700 [00:16<00:00, 152483.04it/s]


Theta & beta estimations on assessmentItemID are completed.
Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'train', 'left_asymptote', 'elo', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'Bigcat', 'Bigcat_avg', 'elapsed',
       'item_avg', 'answer_cnt'],
      dtype='object')


In [10]:
df[FEATS].corr()

,assessmentItemID,KnowledgeTag,Bigcat_avg,item_avg,elapsed,elo,answer_cnt
assessmentItemID,1.000000,-0.066991,-0.908671,-0.482598,-0.007964,-0.397688,-0.250595
KnowledgeTag,-0.066991,1.000000,-0.017651,0.004713,-0.004163,-0.001712,-0.002650
Bigcat_avg,-0.908671,-0.017651,1.000000,0.532841,0.009119,0.433088,0.281175
item_avg,-0.482598,0.004713,0.532841,1.000000,0.059321,0.780177,0.213048
elapsed,-0.007964,-0.004163,0.009119,0.059321,1.000000,0.032196,-0.024029
elo,-0.397688,-0.001712,0.433088,0.780177,0.032196,1.000000,0.388290
answer_cnt,-0.250595,-0.002650,0.281175,0.213048,-0.024029,0.388290,1.000000


In [11]:
def objective(trial):
    params_xgb = {
        "objective": "binary:logistic",
        'metric': 'auc',
        #"eval_metric":'logloss',
        "booster": 'dart',
        'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시
        #"tree_method": 'exact', 'gpu_id': -1,  # CPU 사용시
        #"verbosity": 0,
        #'num_class':3,
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.12),
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000, step=50),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        #"colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        #"reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1),
        #"reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1),
        #'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),     
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 8),
        #"gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True),
        # 'num_parallel_tree': trial.suggest_int("num_parallel_tree", 1, 500) 추가하면 느려짐.
    }
    model = XGBClassifier(**params_xgb)
    model.fit(
        train[FEATS],y_train,
        eval_set=[(train[FEATS], y_train), (test[FEATS], y_test)],
    verbose=50,early_stopping_rounds=100)

    auc_pred = model.predict(test[FEATS])
    auc_score = roc_auc_score(y_test, auc_pred)
    return auc_score


In [12]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="xgb_parameter_opt",
    direction="maximize" # maximize
    # sampler=sampler,
)
study.optimize(objective, n_trials=100)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

/opt/conda/envs/dkt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2022-12-05 18:07:03,013] A new study created in memory with name: xgb_parameter_opt
/tmp/ipykernel_87457/1472279724.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.12),
/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[18:07:05] WARNING: ../src/learner.cc:767: 
Parameters: { "metric" } are not used.

[0]	validation_0-logloss:0.68280	validation_1-logloss:0.68523
[50]	validation_0-logloss:0.49640	validation_1-logloss:0.53937
[100]	validation_0-logloss:0.47098	validation_1-logloss:0.51654
[150]	validation_0-logloss:0.46515	validation_1-logloss:0.51063
[200]	validation_0-logloss:0.46301	validation_1-logloss:0.50885
[250]	validation_0-logloss:0.46195	validation_1-logloss:0.50776
[300]	validation_0-logloss:0.46131	validation_1-logloss:0.50712
[350]	validation_0-logloss:0.46084	validation_1-logloss:0.50679
[400]	validation_0-logloss:0.46048	validation_1-logloss:0.50641
[450]	validation_0-logloss:0.46019	validation_1-logloss:0.50620
[500]	validation_0-logloss:0.45988	validation_1-logloss:0.50593
[550]	validation_0-logloss:0.45966	validation_1-logloss:0.50581
[600]	validation_0-logloss:0.45946	validation_1-logloss:0.50581
[650]	validation_0-logloss:0.45925	validation_1-logloss:0.50579
[700]	validation_0-logl

[I 2022-12-05 18:15:10,310] Trial 0 finished with value: 0.7595557543231961 and parameters: {'max_depth': 4, 'learning_rate': 0.030386123401223025, 'n_estimators': 1000, 'colsample_bytree': 0.7714224803582734, 'min_child_weight': 7}. Best is trial 0 with value: 0.7595557543231961.
/tmp/ipykernel_87457/1472279724.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.12),
/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[18:15:11] WARNING: ../src/learner.cc:767: 
Parameters: { "metric" } are not used.

[0]	validation_0-logloss:0.68311	validation_1-logloss:0.68504
[50]	validation_0-logloss:0.48992	validation_1-logloss:0.53118
[100]	validation_0-logloss:0.46125	validation_1-logloss:0.50958
[150]	validation_0-logloss:0.45515	validation_1-logloss:0.50567
[200]	validation_0-logloss:0.45247	validation_1-logloss:0.50442
[250]	validation_0-logloss:0.45074	validation_1-logloss:0.50360
[299]	validation_0-logloss:0.44946	validation_1-logloss:0.50326


[I 2022-12-05 18:16:20,686] Trial 1 finished with value: 0.7579039952295766 and parameters: {'max_depth': 9, 'learning_rate': 0.02740505657475351, 'n_estimators': 300, 'colsample_bytree': 0.8127750745187065, 'min_child_weight': 2}. Best is trial 0 with value: 0.7595557543231961.
/tmp/ipykernel_87457/1472279724.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.12),
/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

[20:45:17] WARNING: ../src/learner.cc:767: 
Parameters: { "metric" } are not used.

[0]	validation_0-logloss:0.67418	validation_1-logloss:0.67787
[50]	validation_0-logloss:0.46399	validation_1-logloss:0.51096
[100]	validation_0-logloss:0.45590	validation_1-logloss:0.50562
[150]	validation_0-logloss:0.45364	validation_1-logloss:0.50489
[200]	validation_0-logloss:0.45225	validation_1-logloss:0.50449
[250]	validation_0-logloss:0.45115	validation_1-logloss:0.50388
[299]	validation_0-logloss:0.45011	validation_1-logloss:0.50365


[I 2022-12-05 20:46:21,444] Trial 98 finished with value: 0.7623211091234346 and parameters: {'max_depth': 8, 'learning_rate': 0.052836285274305395, 'n_estimators': 300, 'colsample_bytree': 0.7753992220267462, 'min_child_weight': 7}. Best is trial 91 with value: 0.7658825283243889.
/tmp/ipykernel_87457/1472279724.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.12),
/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20:46:22] WARNING: ../src/learner.cc:767: 
Parameters: { "metric" } are not used.

[0]	validation_0-logloss:0.66923	validation_1-logloss:0.67393
[50]	validation_0-logloss:0.45666	validation_1-logloss:0.50669
[100]	validation_0-logloss:0.45109	validation_1-logloss:0.50429
[150]	validation_0-logloss:0.44864	validation_1-logloss:0.50396
[200]	validation_0-logloss:0.44656	validation_1-logloss:0.50410
[249]	validation_0-logloss:0.44468	validation_1-logloss:0.50357


[I 2022-12-05 20:47:10,074] Trial 99 finished with value: 0.7637745974955277 and parameters: {'max_depth': 9, 'learning_rate': 0.0665307468228167, 'n_estimators': 250, 'colsample_bytree': 0.8904113550442735, 'min_child_weight': 7}. Best is trial 91 with value: 0.7658825283243889.


Best Score: 0.7658825283243889
Best trial: {'max_depth': 8, 'learning_rate': 0.08688962700688713, 'n_estimators': 200, 'colsample_bytree': 0.8610908189548545, 'min_child_weight': 7}


In [13]:
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.


Best Score: 0.7657692307692308
Best trial: {'max_depth': 9, 'learning_rate': 0.13614533338992976, 'n_estimators': 620, 'min_child_weight': 6}

Best Score: 0.7637745974955277
Best trial: {'max_depth': 8, 'learning_rate': 0.11002818911337489, 'n_estimators': 350, 'min_child_weight': 7}

Best Score: 0.7658825283243889
Best trial: {'max_depth': 8, 'learning_rate': 0.08688962700688713, 'n_estimators': 200, 'colsample_bytree': 0.8610908189548545, 'min_child_weight': 7}

In [15]:
X = df.drop(['answerCode'], axis=1)
y = df['answerCode']

In [18]:
wandb.init(project = "XGBC")
score_train, score_test = [], [] 
# 유저별 분리
fold = StratifiedGroupKFold(n_splits = 5, shuffle =True)
groups = df['userID']
for train_index , test_index in fold.split(X, y, groups):
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]

    model = XGBClassifier(
    booster="dart",
    learning_rate = 0.08688962700688713,
    max_depth=8,
    min_child_weight=7,
    n_estimators=200,
    colsample_bytree = 0.8610908189548545,
    nthread=4,
    random_state=42,
    #enable_categorical=True
    )

    model.fit(X_train[FEATS], y_train,
    eval_set=[(X_test[FEATS] , y_test)],
    early_stopping_rounds=50,verbose=50)

    y_pred_train = model.predict_proba(X_train[FEATS])[:,1]
    y_pred_test = model.predict_proba(X_test[FEATS])[:,1]
    score_train.append(roc_auc_score( y_train,y_pred_train))
    score_test.append(roc_auc_score( y_test,y_pred_test))
    # make predictions on test

    acc = accuracy_score(y_test, np.where(y_pred_test >= 0.5, 1, 0))
    auc = roc_auc_score( y_test,y_pred_test)
    #wandb.xgboost.log_summary(model, save_model_checkpoint=True)
    wandb.log({"valid_accuracy": acc})
    wandb.log({"valid_roc_auc": auc})

print('\n')
print('Mean training AUC:',np.mean(score_train))
print('Mean testing AUC:',np.mean(score_test))

/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.66250
[50]	validation_0-logloss:0.45931
[100]	validation_0-logloss:0.45736
[150]	validation_0-logloss:0.45670
[199]	validation_0-logloss:0.45640


/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.66302
[50]	validation_0-logloss:0.46301
[100]	validation_0-logloss:0.46125
[150]	validation_0-logloss:0.46069
[199]	validation_0-logloss:0.46035


/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.66256
[50]	validation_0-logloss:0.45887
[100]	validation_0-logloss:0.45709
[150]	validation_0-logloss:0.45659
[199]	validation_0-logloss:0.45627


/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.66296
[50]	validation_0-logloss:0.46246
[100]	validation_0-logloss:0.46072
[150]	validation_0-logloss:0.46025
[199]	validation_0-logloss:0.45994


/opt/conda/envs/dkt/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.66252
[50]	validation_0-logloss:0.45809
[100]	validation_0-logloss:0.45628
[150]	validation_0-logloss:0.45575
[199]	validation_0-logloss:0.45548


Mean training AUC: 0.8462902928690166
Mean testing AUC: 0.8409138438639608


In [33]:
df[:2]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,train,left_asymptote,elo,user_correct_answer,user_total_answer,user_acc,Bigcat,Bigcat_avg,elapsed
0,0,5354,A060000001,1,2020-03-24 00:17:11,556,1,0,0.979350,NaN,0,NaN,5,0.711898,0.0
1,0,5355,A060000001,1,2020-03-24 00:17:14,557,1,0,0.970579,1.0,1,1.0,5,0.711898,3.0


In [25]:
pd.options.display.max_columns = 100

In [26]:
df['userID'].nunique()

6698

In [27]:
submission[:2]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,train,left_asymptote,elo,user_correct_answer,user_total_answer,user_acc,Bigcat,Bigcat_avg,elapsed
2267621,3,4965,A050000133,-1,2020-10-26 13:13:57,469,0,0,0.431119,717.0,1035,0.692754,4,0.658649,46.0
2268292,4,7748,A070000146,-1,2020-12-27 02:47:54,781,0,0,0.508981,465.0,670,0.694030,6,0.521167,23.0


In [28]:
df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'train', 'left_asymptote', 'elo', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'Bigcat', 'Bigcat_avg', 'elapsed'],
      dtype='object')

## 4. 훈련 및 검증

## 5. Inference

In [19]:
# # LOAD TESTDATA
# test_csv_file_path = os.path.join(data_dir, 'test_data.csv')
# test_df = pd.read_csv(test_csv_file_path)

# # FEATURE ENGINEERING
# test_df = feature_engineering(test_df)

# # LEAVE LAST INTERACTION ONLY
# test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# submission = submission[FEATS]
submission.head(10)


# # DROP ANSWERCODE
# test_df = test_df.drop(['answerCode'], axis=1)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,train,left_asymptote,elo,user_correct_answer,user_total_answer,user_acc,Bigcat,Bigcat_avg,elapsed,item_avg,answer_cnt
2267621,3,4965,A050000133,-1,2020-10-26 13:13:57,469,0,0,0.431119,717.0,1035,0.692754,4,0.658649,46.0,0.528000,4.0
2268292,4,7748,A070000146,-1,2020-12-27 02:47:54,781,0,0,0.508981,465.0,670,0.694030,6,0.521167,23.0,0.560000,4.0
2269609,13,7484,A070000111,-1,2020-12-27 04:35:09,820,0,0,0.307210,915.0,1316,0.695289,6,0.521167,8.0,0.364000,2.0
2270869,17,9381,A090000064,-1,2020-10-30 05:48:37,309,0,0,0.447042,1031.0,1259,0.818904,8,0.454470,75.0,0.260000,5.0
2271256,26,6231,A060000135,-1,2020-10-23 11:44:18,183,0,0,0.247744,293.0,386,0.759067,5,0.711898,17.0,0.308000,3.0
2272110,29,1967,A020000190,-1,2020-10-22 04:38:45,680,0,0,0.867810,723.0,853,0.847597,1,0.737515,30.0,0.706667,4.0
2273194,45,3759,A040000136,-1,2020-10-23 08:24:19,276,0,0,0.677722,746.0,1083,0.688827,3,0.679714,4.0,0.603333,1.0
2273887,53,3779,A040000140,-1,2020-10-26 09:13:20,279,0,0,0.040234,362.0,692,0.523121,3,0.679714,20.0,0.366667,2.0
2274698,58,7847,A070000159,-1,2020-12-24 21:09:29,792,0,0,0.182091,295.0,810,0.364198,6,0.521167,2.0,0.384000,0.0
2275968,64,7748,A070000146,-1,2020-12-29 04:30:22,781,0,0,0.755090,1058.0,1269,0.833727,6,0.521167,2.0,0.560000,5.0


In [20]:
# MAKE PREDICTION
total_preds = model.predict_proba(submission[FEATS])[:,1]
total_preds[:10]

array([0.64841384, 0.75525796, 0.20916943, 0.63946396, 0.16276938,
       0.8261331 , 0.10122847, 0.13359335, 0.17195348, 0.90329576],
      dtype=float32)

In [21]:
# SAVE OUTPUT
output_dir = 'output/'
write_path = os.path.join(output_dir, "xgbdartkfold_submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/xgbdartkfold_submission.csv


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

